In [ ]:
import cv2
import os
import pandas as pd
from PIL import Image
import numpy as np
import re
import glob

In [ ]:
img_dir = r'test'
search_path = os.path.join(img_dir, "**",  "*.png")
files = glob.glob(search_path, recursive=True)
img_files = [f.replace('\\', '/') for f in files]
print(len(img_files))


3824


In [ ]:
def pad_image(img_arr, height, width):
    # Calculate the amount of padding needed
    h, w = img_arr.shape[:2]
    pad_h = max(0, height - h)
    pad_w = max(0, width - w)

    # Add padding to the image
    padded_arr = np.pad(img_arr, ((0, pad_h), (0, pad_w)), mode='constant', constant_values=0)
    return(padded_arr)


In [ ]:
filename = []
maxh = 310
maxw = 360
image_data = np.zeros((len(img_files), maxh, maxw),dtype=np.uint8)
shape = []

for i,img_path in enumerate(img_files):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # Convert to grayscale
    img_np = np.asarray(img)
    label1 = re.search(r'case[\d]+_day[\d]+', img_path)
    label2 = re.search(r'slice_[\d]+', img_path)
    tmp_list = img_path.split('_')
    shape.append((int(tmp_list[4]), int(tmp_list[3])))
    fn = label1[0]+"_"+label2[0]
    image_data[i] = pad_image(img_np, maxh, maxw)
    filename.append(fn)
    if (i%1000 == 0):
      print(i)

0
1000
2000
3000


In [ ]:
tmp_shape = len(img_files)

In [ ]:
img2_5D = np.zeros((tmp_shape, 3, maxh, maxw), dtype = int)
labels = np.zeros((tmp_shape, 3, maxh, maxw), dtype = int)

In [ ]:
def rle_decode(mask_rle, shape=(768, 768)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return
    Returns numpy array, 1 - mask, 0 - background
    '''
    if not isinstance(mask_rle, str):
        img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
        return img.reshape(shape)

    s = mask_rle.split()
    s = s[1:]
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

In [ ]:
path = r'train/data.csv'
df = pd.read_csv(path)
pd.options.display.max_colwidth = None
j = 0
for i in range(len(img_files)):
    if (i % 1000 ==0):
        print(i, j)
    # process the img2.5D
    if (i == 0 or i == len(img_files) - 1):
        continue
    pre_sn = re.search(r"(\w+_\w+)_slice", filename[i-1]).group(1)
    curr_sn = re.search(r"(\w+_\w+)_slice", filename[i]).group(1)
    next_sn = re.search(r"(\w+_\w+)_slice", filename[i+1]).group(1)
    if (pre_sn.lower() != curr_sn.lower()):
        continue
    if(next_sn.lower() != curr_sn.lower()):
        continue
        
    img2_5D[j,1,:,:] = image_data[i]
    img2_5D[j,0,:,:] = image_data[i-1]
    img2_5D[j,2,:,:] = image_data[i+1]

  # process the masks aka labels
    fn = filename[i]
    valid = 0
    filtered_df = df[(df['id'] == fn) & (df['class'] == 'large_bowel')]
    if (not filtered_df["segmentation"].isna().all()):
        valid = 1
        labels[j,0,:,:] = pad_image(rle_decode(filtered_df["segmentation"].to_string(), shape[i]), maxh, maxw)
    
    filtered_df = df[(df['id'] == fn) & (df['class'] == 'small_bowel')]
    if (not filtered_df["segmentation"].isna().all()):
        valid = 1
        labels[j,1,:,:] = pad_image(rle_decode(filtered_df["segmentation"].to_string(), shape[i]), maxh, maxw)
    
    filtered_df = df[(df['id'] == fn) & (df['class'] == 'stomach')]
    if (not filtered_df["segmentation"].isna().all()):
        valid = 1
        labels[j,2,:,:] = pad_image(rle_decode(filtered_df["segmentation"].to_string(), shape[i]), maxh, maxw)
    
    if (valid):
        j = j + 1
    if (j >= 7000): 
        break

0 0
1000 407
2000 765
3000 1148


In [ ]:
img2_5D = img2_5D[:j, :,:,:]
labels = labels[:j, :, :, :]

In [ ]:
print(img2_5D.shape, labels.shape)
np.save("train/test_data.npy", img2_5D)
np.save("train/test_labels.npy", labels)

(1471, 3, 310, 360) (1471, 3, 310, 360)


In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(3, 3, figsize=(15, 10))
k = 1000
for i in range(3):
    ax[0, i].imshow(labels[3*k + 0,i,:,:], cmap='gray')
    ax[0, i].set_title(f"True Label (Channel {i})")
    
    ax[1, i].imshow(labels[3*k + 1,i,:,:], cmap='gray')
    ax[1, i].set_title(f"True Label (Channel {i})")
    
    ax[2, i].imshow(labels[3*k + 2,i,:,:], cmap='gray')
    ax[2, i].set_title(f"True Label (Channel {i})")

IndexError: index 3000 is out of bounds for axis 0 with size 1471

In [ ]:
training_data, val_data, test_data = img2_5D[training_idx, :,:,:], img2_5D[val_idx, :,:,:], img2_5D[test_idx, :,:,:] 
training_label, val_label, test_label = labels[training_idx, :,:,:], labels[val_idx, :,:,:], labels[test_idx, :,:,:] 

In [ ]:
print(training_data.shape, val_data.shape, test_data.shape)
print(training_label.shape, val_label.shape, test_label.shape)

In [ ]:
np.save("train/training_data.npy", training_data)
np.save("train/val_data.npy", val_data)
np.save("train/test_data.npy", test_data)

In [ ]:
np.save("train/training_label.npy", training_label)
np.save("train/val_label.npy", val_label)
np.save("train/val_label.npy", test_label)